# Microbiota analysis
## Author: Tijs van Lieshout

### Import statements:

In [1]:
import pandas as pd
import glob
from IPython.display import display
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [22]:
def get_config():
    '''
    Function that gets the configuration of personalized variable definitions
    '''
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

### Loading the data:

In [2]:
def load_data_microbiota(PATH):
    """Load microbiota data (Gut Feeling Knowledge Base and metaphlann output) into pandas 
    dataframes

    Keyword arguments:
    PATH -- The path which contains the Gut Feeling Knowledge Base and metaphlann output dir
    
    Returns:
    gfkb -- A pandas dataframe containing the Gut Feeling Knowledge Base
    tax_profiles -- A pandas dataframe containing the taxonomic profile per barcode for all barcodes"""
    
    gfkb = pd.read_csv(f"{PATH}/GutFeelingKnowledgeBase-v4-Master_List.csv")
    gfkb = gfkb.drop(columns=["Present in GFKB v3 (Y/N)",
                              "Present in GFKB_epilepsy v3 (Y/N)"])
    gfkb = pd.concat([gfkb.drop(columns=["Genome Size (Mb)"]).apply(lambda x: x.astype(str)), 
                      gfkb["Genome Size (Mb)"]], axis=1)

    tax_profiles = pd.DataFrame(columns=["clade_name", 
                                         "NCBI_tax_id", 
                                         "relative_abundance", 
                                         "additional_species",
                                         "barcode"])
    
    for file in glob.glob(f"{PATH}/metaphlan_output/*.txt"):
        tax_profile = pd.read_csv(file, 
                                  comment="#", 
                                  sep="\t", 
                                  names=["clade_name", 
                                         "NCBI_tax_id", 
                                         "relative_abundance", 
                                         "additional_species"])
        
        barcode = file.split('metaphlan_output/')[1].split("_all")[0]
        tax_profile['barcode'] = barcode
        tax_profile = tax_profile.set_index([tax_profile.barcode, tax_profile.clade_name]).sort_index()
        tax_profiles = pd.concat([tax_profiles,
                                  tax_profile])

    tax_profiles = tax_profiles.set_index([tax_profiles.barcode, tax_profiles.clade_name]).sort_index().drop(columns=['clade_name','barcode'])
    return gfkb, tax_profiles

In [87]:
def select_on_tax_level(tax_profiles, tax_level = "species"):
    if tax_level == "kingdom":
        included = r"k__"
        not_included = r"p__"
    elif tax_level == "phylum":
        included = r"p__"
        not_included = r"c__"
    elif tax_level == "class":
        included = r"c__"
        not_included = r"o__"
    elif tax_level == "order":
        included = r"o__"
        not_included = r"f__"
    elif tax_level == "family":
        included = r"f__"
        not_included = r"g__"
    elif tax_level == "genus":
        included = r"g__"
        not_included = "s__"
    elif tax_level == r"species":
        included = r"s__"
        not_included = r"x__"
    
    return tax_profiles[(tax_profiles.index.get_level_values('clade_name').str.contains(included, regex=True) == True) &
                        (tax_profiles.index.get_level_values('clade_name').str.contains(not_included, regex=True) == False)]

### Recreating the plot from Zimmer et al. 2012
<img src="../microbiota_tax_data/zimmer_species_abundance_plot.png" alt="Zimmer et al. 2012" width="400"/>

In [3]:
def recreate_zimmer(tax_profiles):
    """recreate a comparison of taxa that have been routinely analysed by Zimmer et al. 2012
    Keyword arguments:
    tax_profiles -- A pandas datatax_profilesframe containing the taxonomic profile per barcode for all barcodes
    
    Returns:
    zimmer_subset -- A pandas dataframe containing a subset of taxa of interest of the taxonomic profile per barcode for all barcodes 
    dataframes containing only the taxa analyzed by zimmer et al. 2012 as values
    """
    # Subset for the Zimmer et al. 2012 bar plot
    bacteroides_subset = tax_profiles[tax_profiles.index.get_level_values('clade_name').str.endswith("g__Bacteroides")]
    bifidobacteria_subset = tax_profiles[tax_profiles.index.get_level_values('clade_name').str.endswith("g__Bifidobacterium")]
    ecoli_subset = tax_profiles[tax_profiles.index.get_level_values('clade_name').str.endswith("s__Escherichia_coli")]
    enterobacter_subset = tax_profiles[tax_profiles.index.get_level_values('clade_name').str.endswith("f__Enterobacteriaceae")]

    # Other taxa Zimmer et al. 2012 deemed of interest
    clostridium_subset = tax_profiles[tax_profiles.index.get_level_values('clade_name').str.contains("Clostridium")]

    zimmer_subset = pd.concat([bacteroides_subset,
                               bifidobacteria_subset,
                               ecoli_subset,
                               enterobacter_subset]).sort_index()

#     for taxa in clostridium_subset.clade_name:
#         print(taxa)
        
    return zimmer_subset

In [91]:
def main():
    PATH = "../microbiota_tax_data"
    gfkb, tax_profiles = load_data_microbiota(PATH)

    barcode2zimmer_subset = recreate_zimmer(tax_profiles)
    with open("tmp.html",'w') as f:
        f.write(tax_profiles.to_html(justify="left"))
    display(select_on_tax_level(tax_profiles, "species").info())
    
if __name__ == '__main__':
    main()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 109 entries, ('barcode_01', 'k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Bifidobacteriales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium_adolescentis') to ('barcode_10', 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii')
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NCBI_tax_id         109 non-null    object 
 1   relative_abundance  109 non-null    float64
 2   additional_species  65 non-null     object 
dtypes: float64(1), object(2)
memory usage: 9.5+ KB


None